In [20]:
!pwd
%cd /content/drive/MyDrive/colab
!ls -al

/content/drive/MyDrive/colab
/content/drive/MyDrive/colab
total 29
drwx------ 3 root root  4096 May 20 14:57  box_image
drwx------ 3 root root  4096 May 22 07:29  data
-rw------- 1 root root 12773 Jun 10 08:22 'faster r-cnn 예제.ipynb'
drwx------ 7 root root  4096 Jun 10 08:20  .git
drwx------ 2 root root  4096 May 18 13:29  .ipynb_checkpoints


In [8]:
!git init

Initialized empty Git repository in /content/drive/My Drive/colab/.git/


In [32]:
!git commit -am "first"

[master (root-commit) 765aeb1] first
 16 files changed, 1 insertion(+)
 create mode 100644 box_image/box_img.jpg
 create mode 100644 box_image/box_img0.jpg
 create mode 100644 box_image/box_img1.jpg
 create mode 100644 box_image/box_img2.jpg
 create mode 100644 box_image/box_img3.jpg
 create mode 100644 box_image/box_img4.jpg
 create mode 100644 box_image/box_img5.jpg
 create mode 100644 box_image/box_img6.jpg
 create mode 100644 data/image0.jpg
 create mode 100644 data/image1.jpg
 create mode 100644 data/image2.jpg
 create mode 100644 data/image3.jpg
 create mode 100644 data/image4.jpg
 create mode 100644 data/image5.jpg
 create mode 100644 data/image6.jpg
 create mode 100644 "faster r-cnn \341\204\213\341\205\250\341\204\214\341\205\246.ipynb"


In [42]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   "faster r-cnn \341\204\213\341\205\250\341\204\214\341\205\246.ipynb"

no changes added to commit (use "git add" and/or "git commit -a")


In [43]:
import tensorflow as tf

import tensorflow_hub as hub

# 이미지 그리기 위함
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

import cv2

import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# 시간 측정
import time

# 할당된 gpu 
print(tf.test.gpu_device_name())

/device:GPU:0


# 새 섹션

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

#http url로 이미지를 받을경우 함수
def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename
#내부 posix주소로 파일을 받을 경우 함수
def download_and_resize_image2(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  path=url
  img=Image.open(path)
  img_resize=img.resize((new_width,new_height))
  img_resize.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(img_resize)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
 #이미지에 바운딩 박스 그리기
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)
 
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  #각 디스플레이 문자열은 0.05배
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)
  # 문자의 높이가 바운딩방스의 위에를 초과하면 바운딩박스 밑에 문자 추가
  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # 리스트 출력
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

# 탐지할 박스의 개수 및 최소확률 정의 함수.
def draw_boxes(image, boxes, class_names, scores, max_boxes=40, min_score=0.3):
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
image_list=[]
downloaded_image_path_list=[]
for i in range(7):
  image_list.append(f"/content/drive/MyDrive/colab/data/image{i}.jpg")
#image_url = "/content/drive/MyDrive/colab/data/image0.jpg"  #@param
for i in range(7):
  downloaded_image_path = download_and_resize_image2(image_list[i], 1280, 960, True)
  downloaded_image_path_list.append(downloaded_image_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### run 함수

In [ ]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("%d 물체를 찾았다." % len(result["detection_scores"]))
  print("실행 시간: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)
  return image_with_boxes

### 바운더리 서치 시행

In [ ]:
iwb=[]
for i in range(7):
  iwb.append(run_detector(detector, downloaded_image_path_list[i]))

Output hidden; open in https://colab.research.google.com to view.

## PIL이미지로 전환 및 드라이브 저장

In [ ]:
for i in range(7):
  nti=Image.fromarray(iwb[i],'RGB')
  nti.save(f"/content/drive/MyDrive/colab/box_image/box_img{i}.jpg")